In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用芝加哥出租车行程数据集进行出租车费用预测

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/chicago_taxi_fare_prediction/chicago_taxi_fare_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/chicago_taxi_fare_prediction/chicago_taxi_fare_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/chicago_taxi_fare_prediction/chicago_taxi_fare_prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本笔记本演示了在 Vertex AI 上配置可解释 AI 的分析、特征选择、模型构建和部署，使用芝加哥出租车行程数据集的子集进行出租车费用预测。

*注意：此笔记本是为在 [Vertex AI Workbench 管理的笔记本](https://console.cloud.google.com/vertex-ai/workbench/list/managed)实例中使用 Python (Local) 内核而开发的。本笔记本的某些组件在其他笔记本环境中可能无法正常工作。*

了解有关 [Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction) 和 [Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview) 的更多信息。

### 目标

这个笔记本的目标是通过尝试解决出租车票价预测问题，提供有关最新Vertex AI功能（如**可解释AI**和**笔记本中的BigQuery**）的概述。

本教程使用以下Google Cloud ML服务和资源：

- `Vertex AI模型`资源
- `Vertex AI端点`资源
- `Vertex可解释AI`
- Google Cloud存储
- BigQuery

执行的步骤包括：

- 使用“笔记本中的BigQuery”加载数据集。
- 对数据集进行探索性数据分析。
- 特征选择和预处理。
- 使用scikit-learn构建线性回归模型。
- 配置模型以使用Vertex可解释AI。
- 将模型部署到Vertex AI。
- 测试部署的模型。
- 清理。

### 数据集

芝加哥出租车行程数据集包括2013年至今报告给芝加哥市作为监管机构的出租车行程。为了保护隐私但允许进行聚合分析，出租车ID对于任何给定的出租车牌号是一致的，但不显示号码，人口普查区在某些情况下会被抑制，时间会被舍入到最近的15分钟。由于数据报告的过程，不是所有的行程都被报告，但市政府认为大部分都被报告了。这个数据集可以在BigQuery上作为一个公共数据集使用，表ID为`bigquery-public-data.chicago_taxi_trips.taxi_trips`，也可以在Kaggle上作为一个公共数据集使用，网址为[芝加哥出租车行程](https://www.kaggle.com/chicago/chicago-taxi-trips-bq)。

有关此数据集及其创建方式的更多信息，请参阅[芝加哥数字网站](http://digital.cityofchicago.org/index.php/chicago-taxi-data-released)。

本教程考虑的原始数据集很大且嘈杂，因此将使用特定日期范围内的数据。根据各种在线资源，2018年5月左右的数据与其他日期范围相比取得了一些非常好的结果。虽然也有一些复杂的模型用于同一问题，比如考虑天气数据、节假日和季节等，但当前笔记本仅探讨一个简单的线性回归模型。我们的主要目标是演示在Vertex AI上配置了Vertex Explainable AI的模型部署。

所选数据集包括以下字段：

- `unique_key`：行程的唯一标识符。
- `taxi_id`：出租车的唯一标识符。
- `trip_start_timestamp`：行程开始时间，舍入到最近的15分钟。
- `trip_end_timestamp`：行程结束时间，舍入到最近的15分钟。
- `trip_seconds`：行程时间，以秒为单位。
- `trip_miles`：行程距离，以英里为单位。
- `pickup_census_tract`：行程开始的人口普查区。为了保护隐私，这些人口普查区对于一些行程是不可见的。
- `dropoff_census_tract`：行程结束的人口普查区。为了保护隐私，这些人口普查区对于一些行程是不可见的。
- `pickup_community_area`：行程开始的社区区域。
- `dropoff_community_area`：行程结束的社区区域。
- `fare`：行程费用。
- `tips`：小费金额。现金小费通常不会记录。
- `tolls`：行程的通行费。
- `extras`：行程的额外费用。
- `trip_total`：行程总费用，包括车费、小费、通行费和额外费用的总和。
- `payment_type`：行程的支付方式。
- `company`：出租车公司。
- `pickup_latitude`：起点人口普查区的纬度，或者如果人口普查区由于隐私原因被隐藏，则为社区区域的纬度。
- `pickup_longitude`：起点人口普查区的经度，或者如果人口普查区由于隐私原因被隐藏，则为社区区域的经度。
- `pickup_location`：起点人口普查区的位置，或者如果人口普查区由于隐私原因被隐藏，则为社区区域的位置。
- `dropoff_latitude`：终点人口普查区的纬度，或者如果人口普查区由于隐私原因被隐藏，则为社区区域的纬度。
- `dropoff_longitude`：终点人口普查区的经度，或者如果人口普查区由于隐私原因被隐藏，则为社区区域的经度。
- `dropoff_location`：终点人口普查区的位置，或者如果人口普查区由于隐私原因被隐藏，则为社区区域的位置。

成本

本教程使用Google Cloud的以下计费组件：

- Vertex AI
- BigQuery
- Cloud Storage

了解[Vertex AI的价格](https://cloud.google.com/vertex-ai/pricing)，[BigQuery的价格](https://cloud.google.com/bigquery/pricing)和[Cloud Storage的价格](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

安装以下必要的包以执行这个笔记本。

In [ ]:
! pip3 install --quiet --upgrade    google-cloud-bigquery \
                                    google-cloud-aiplatform \
                                    google-cloud-storage \
                                    seaborn==0.12.0 \
                                    scikit-learn \
                                    pandas \
                                    fsspec \
                                    db-dtypes \
                                    pyarrow   

只有协作才能：取消下面的单元格注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参阅支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

将地区设置

**可选项**：更新“REGION”变量以指定要使用的地区。深入了解[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行直播教程，您可能会使用共享的测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of length 8
def generate_uuid():
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=8))


UUID = generate_uuid()

### 验证您的Google Cloud帐号

要验证您的Google Cloud帐号，请按照您的Jupyter环境的说明进行操作：

**1. Vertex AI Workbench**
<br>您已经通过身份验证。

**2. 本地JupyterLab实例**
<br>取消注释并运行以下代码：

In [ ]:
# ! gcloud auth login

取消注释并运行以下代码：

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上授予您的服务帐号Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**只有你的存储桶不存在时**：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import os
import pickle

import matplotlib.pyplot as plt
# load the required libraries
import pandas as pd
import seaborn as sns
from google.cloud import aiplatform, storage
from google.cloud.aiplatform_v1.types import SampledShapleyAttribution
from google.cloud.aiplatform_v1.types.explanation import ExplanationParameters
from google.cloud.bigquery import Client

%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

## 通过BigQuery集成访问数据

Vertex AI Workbench托管笔记本的**BigQuery集成**功能允许您直接从笔记本中使用BigQuery及其功能，无需每次在选项卡之间切换。在笔记本的每个单元格中，都有一个位于右上角的BigQuery集成选项，选择它将使您能够编写一个在BigQuery中执行的SQL查询。

在数据集中可用的字段中，只选择那些对分析和建模有用的看起来普遍和相关的字段，如`taxi_id`、`trip_start_timestamp`、`trip_seconds`、`trip_miles`、`payment_type`和`trip_total`。此外，`trip_total`字段被视为机器学习模型将预测的目标变量。显然，该字段是`fare`、`tips`、`tolls`和`extras`字段的总和，因此由于它们与目标变量的相关性，它们被排除在建模之外。由于数据量很大，考虑了在2018年5月12日至2018年5月18日期间的数据集子集。在此日期范围内，数据点可能会有噪音，因此考虑了一些条件，如下所示：

- 行程所用时间> 0。
- 行程中行驶的距离> 0。
- 总行程费用> 0。
- 上下车的地点是有效的（不为空）。

注意：下面的单元格是一个BigQuery集成单元格，仅可在Vertex AI Workbench托管实例上执行。如果您的笔记本环境不同，您可以跳过此单元格。

#@bigquery

选择 
-- 选择所需字段
taxi_id, trip_start_timestamp, 
trip_seconds, trip_miles, trip_total, 
payment_type

从 `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
其中 
-- 指定所需准则
trip_start_timestamp >= '2018-05-12' 并且 
trip_end_timestamp <= '2018-05-18' 并且
trip_seconds > 0 并且
trip_miles > 0 并且
trip_total > 3 并且
pickup_community_area 不是 NULL 并且 
dropoff_community_area 不是 NULL

BigQuery集成还允许您使用“查询和加载为DataFrame”按钮将查询的数据加载到pandas dataframe中。 点击该按钮会添加一个新的单元格，其中提供了一个代码片段，可将数据加载到dataframe中。

选择所需的字段####

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.

client = Client(project=PROJECT_ID)

query = """select 
taxi_id, trip_start_timestamp, 
trip_seconds, trip_miles, trip_total, 
payment_type, pickup_community_area, 
dropoff_community_area 

from `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
where 
trip_start_timestamp >= '2018-05-12' and 
trip_end_timestamp <= '2018-05-18' and
trip_seconds > 60 and trip_seconds < 6*60*60 and
trip_miles > 0 and
trip_total > 3 and
pickup_community_area is not NULL and 
dropoff_community_area is not NULL"""
job = client.query(query)
df = job.to_dataframe()

检查数据中的字段及其形状。

In [ ]:
# check the dataframe's shape
print(df.shape)
# check the columns in the dataframe
df.columns

检查一些样本数据。

In [ ]:
df.head()

检查数据中字段的数据类型。

In [ ]:
df.dtypes

检查数据框中的空值。

In [ ]:
df.info()

根据数据中空值的百分比，可以选择删除它们或者用均值/中位数（对于数值）和众数（对于分类值）填充它们。在当前数据中，似乎没有任何空值。

#### 检查字段的数值分布（数值）。

如果有任何具有恒定值的字段，那些字段可以被删除，因为它们对模型没有任何价值。

In [ ]:
df.describe().T

在当前数据集中，`trip_total` 是目标字段。为了轻松地通过它们的类型访问字段，请识别数据中的分类和数值字段，并保存它们。

In [ ]:
target = "trip_total"
categ_cols = ["payment_type", "pickup_community_area", "dropoff_community_area"]
num_cols = ["trip_seconds", "trip_miles"]

进一步分析数据时，可以使用各种图表来处理数值和分类字段。对于数值数据，可以使用直方图和箱线图。柱状图适用于分类数据，以更好地了解数据的分布和异常值。

#### 在数值字段上绘制直方图和箱线图。

In [ ]:
for i in num_cols + [target]:
    _, ax = plt.subplots(1, 2, figsize=(12, 4))
    df[i].plot(kind="hist", bins=100, ax=ax[0])
    ax[0].set_title(str(i) + " -Histogram")
    df[i].plot(kind="box", ax=ax[1])
    ax[1].set_title(str(i) + " -Boxplot")
    plt.show()

字段`trip_seconds`描述了行程所花费的时间（以秒为单位）。为了方便我们的分析，让我们将其转换为小时。

In [ ]:
df["trip_hours"] = round(df["trip_seconds"] / 3600, 2)
df["trip_hours"].plot(kind="box")

同样，可以通过将`trip_miles`和`trip_hours`相除来添加另一个字段`trip_speed`，以了解行程的时速（英里/小时）。

In [ ]:
df["trip_speed"] = round(df["trip_miles"] / df["trip_hours"], 2)
df["trip_speed"].plot(kind="box")

迄今为止，您只查看了单变量图。为了更好地理解变量之间的关系，可以绘制一对图。

In [ ]:
# generate a pairplot for 10K samples
try:
    sns.pairplot(
        data=df[["trip_seconds", "trip_miles", "trip_total", "trip_speed"]].sample(
            10000
        )
    )
    plt.show()
except Exception as e:
    print(e)

从到目前为止可视化的箱线图和直方图可以看出，数据中有一些离群值引起了偏斜，也许可以将其删除。此外，在成对绘图中考虑的自变量之间可以看到一些线性关系。例如，`trip_seconds`和`trip_miles`以及因变量`trip_total`之间存在线性关系。

根据以下条件限制数据，以在一定程度上去除数据中的异常值：
- 总费用至少超过3美元。
- 总行驶里程大于0且小于300英里。
- 总行驶秒数至少为1分钟。
- 总行驶小时数不超过2小时。
- 行程速度不超过70英里/小时。

这些条件是基于一些一般假设，因为很明显存在一些记录错误，例如速度大于500英里/小时和旅行时间超过5小时，导致数据中的异常值。

In [ ]:
# set constraints to remove outliers
df = df[df["trip_total"] > 3]

df = df[(df["trip_miles"] > 0) & (df["trip_miles"] < 300)]

df = df[df["trip_seconds"] >= 60]

df = df[df["trip_hours"] <= 2]

df = df[df["trip_speed"] <= 70]
df.reset_index(drop=True, inplace=True)
df.shape

## 分析分类数据

#### 此外，可以通过绘制每个字段中所有级别的分布来进一步探索分类数据。

In [ ]:
for i in categ_cols:
    print(df[i].unique().shape)
    df[i].value_counts(normalize=True).plot(kind="bar", figsize=(10, 4))
    plt.title(i)
    plt.show()

通过上面的分析，可以看到几乎99%的交易类型是现金和信用卡。虽然还有其他类型的交易，但它们的分布可以忽略不计。在这种情况下，可以放弃较低的分布级别。另一方面，接送社区区域的总数量似乎都处于相同的水平，这是合理的。在这种情况下，也可以选择省略较低的分布级别，但必须确保之后两个字段的级别是相同的。在当前的笔记本中，保持它们原样，并继续建模。

目标变量与分类字段之间的关系可以通过箱线图来表示。可以通过每个级别来识别目标变量的相应分布。

In [ ]:
for i in categ_cols:
    plt.figure(figsize=(10, 4))
    sns.boxplot(x=i, y=target, data=df)
    plt.xticks(rotation=45)
    plt.title(i)
    plt.show()

有一个案例中`trip_total`超过3000，并且有相同的接送区域：28显然是与其他点相比的异常值。这个数据点可以被移除。

In [ ]:
df = df[df["trip_total"] < 3000].reset_index(drop=True)

只保留“信用卡”和“现金”支付类型。另外，通过将0分配给“信用卡”和1分配给“现金”支付类型来对它们进行编码。

In [ ]:
# add payment_type
df = df[df["payment_type"].isin(["Credit Card", "Cash"])].reset_index(drop=True)
# encode the payment types
df["payment_type"] = df["payment_type"].apply(
    lambda x: 0 if x == "Credit Card" else (1 if x == "Cash" else None)
)

数据中还有一些有用的时间戳字段。`trip_start_timestamp`代表出租车行程的开始时间戳，可以从中推导出它是一周的哪一天以及是哪个小时。

In [ ]:
df["trip_start_timestamp"] = pd.to_datetime(df["trip_start_timestamp"])
df["dayofweek"] = df["trip_start_timestamp"].dt.dayofweek
df["hour"] = df["trip_start_timestamp"].dt.hour

由于当前数据集仅限于一周，如果新推导字段与目标变量的变化不大，它们可以被舍弃。

#### 绘制`trip_total`的总和和平均值，与`dayofweek`相关。

In [ ]:
# plot sum and average of trip_total w.r.t the dayofweek
_, ax = plt.subplots(1, 2, figsize=(10, 4))
df[["dayofweek", "trip_total"]].groupby("dayofweek").trip_total.sum().plot(
    kind="bar", ax=ax[0]
)
ax[0].set_title("Sum of trip_total")
df[["dayofweek", "trip_total"]].groupby("dayofweek").trip_total.mean().plot(
    kind="bar", ax=ax[1]
)
ax[1].set_title("Avg. of trip_total")
plt.show()

#### 计算`trip_total`关于`hour`的总和和平均值。

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 4))
df[["hour", "trip_total"]].groupby("hour").trip_total.sum().plot(kind="bar", ax=ax[0])
ax[0].set_title("Sum of trip_total")
df[["hour", "trip_total"]].groupby("hour").trip_total.mean().plot(kind="bar", ax=ax[1])
ax[1].set_title("Avg. of trip_total")
plt.show()

由于这些情节在目标变量的各个级别上似乎没有恒定的数字，因此可以考虑将其用于训练。实际上，为了简化事务，这些派生特征可以合并为较少的级别。

“dayofweek”字段可以被合并为一个二进制字段，考虑到是否为周末。如果是工作日，则可以分配1，否则为0。同样，“hour”字段也可以被划分和编码。芝加哥的正常工作时间可以假定在上午8点至晚上10点之间，如果数值在工作时间内，则编码为1，否则为0。

In [ ]:
# bucket and encode the dayofweek and hour
df["dayofweek"] = df["dayofweek"].apply(lambda x: 0 if x in [5, 6] else 1)
df["hour"] = df["hour"].apply(lambda x: 0 if x in [23, 0, 1, 2, 3, 4, 5, 6, 7] else 1)

在训练模型之前，请检查数据分布。

In [ ]:
df.describe().T

## 将数据分成训练集和测试集

#### 将经过预处理的数据集分成训练集和测试集，以便线性回归模型可以在测试集上进行验证。

In [ ]:
cols = [
    "trip_seconds",
    "trip_miles",
    "payment_type",
    "pickup_community_area",
    "dropoff_community_area",
    "dayofweek",
    "hour",
    "trip_speed",
]
x = df[cols].copy()
y = df[target].copy()

# split the data into 75-25% ratio
X_train, X_test, y_train, y_test = train_test_split(
    x, y, train_size=0.75, test_size=0.25, random_state=13
)
X_train.shape, X_test.shape

## 拟合一个简单的线性回归模型

#### 使用scikit-learn的LinearRegression方法在训练数据上拟合一个线性回归模型。

In [ ]:
# Building the regression model
reg = LinearRegression()
reg.fit(X_train, y_train)

打印模型在训练集和测试集上的 `R2分数` 和 `RMSE` 值。

In [ ]:
# print test R2 score
y_train_pred = reg.predict(X_train)
train_score = r2_score(y_train, y_train_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
y_test_pred = reg.predict(X_test)
test_score = r2_score(y_test, y_test_pred)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print("Train R2-score:", train_score, "Train RMSE:", train_rmse)
print("Test R2-score:", test_score, "Test RMSE:", test_rmse)

一个低的RMSE误差和0.93的训练和测试R2得分表明模型拟合很好。此外，还可以通过检查sklearn模型的`coef_`属性来检查模型学习的每个独立变量的系数。

#### 检查模型学习的系数。

In [ ]:
coef_df = pd.DataFrame({"col": cols, "coeff": reg.coef_})
coef_df.set_index("col").plot(kind="bar")

保存模型并上传到云存储桶

要在Vertex AI上部署模型，模型文件需要首先存储在云存储桶中。

In [ ]:
FILE_NAME = "model.pkl"
with open(FILE_NAME, "wb") as file:
    pickle.dump(reg, file)

# Upload the saved model file to Cloud Storage
BLOB_PATH = "taxicab_fare_prediction/"

BLOB_NAME = BLOB_PATH + FILE_NAME

bucket = storage.Client().bucket(BUCKET_URI[5:])
blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(FILE_NAME)

使用支持Vertex Explainable AI的Vertex AI在模型上部署

在部署模型之前配置Vertex Explainable AI。了解有关在Vertex AI模型中配置Vertex Explainable AI的更多信息，请访问链接：https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations#scikit-learn-and-xgboost-pre-built-containers。

In [ ]:
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type: "string"}

In [ ]:
# If the model display name is not set, choose the default one
if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "taxi_fare_prediction_model"


ARTIFACT_GCS_PATH = f"{BUCKET_URI}/{BLOB_PATH}"

# Feature-name(Inp_feature) and Output-name(Model_output) can be arbitrary
exp_metadata = {"inputs": {"Input_feature": {}}, "outputs": {"Predicted_taxi_fare": {}}}

#### 使用配置的解释元数据从上传的模型创建一个模型资源。

In [ ]:
# Create a Vertex AI model resource with support for Vertex Explainable AI

aiplatform.init(project=PROJECT_ID, location=REGION)

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_GCS_PATH,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",
    explanation_metadata=exp_metadata,
    explanation_parameters=ExplanationParameters(
        sampled_shapley_attribution=SampledShapleyAttribution(path_count=25)
    ),
)

model.wait()

print(model.display_name)
print(model.resource_name)

### 为模型创建一个端点资源

#### 为端点设置一个显示名称并创建端点资源。

In [ ]:
ENDPOINT_DISPLAY_NAME = "[your-endpoint-display-name]"  # @param {type: "string"}

# If the display name is not set, choose the default one
if ENDPOINT_DISPLAY_NAME == "[your-endpoint-display-name]":
    ENDPOINT_DISPLAY_NAME = "taxi_fare_prediction_endpoint"

endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME, project=PROJECT_ID, location=REGION
)

print(endpoint.display_name)
print(endpoint.resource_name)

### 使用所需的机器类型将模型部署到创建的终端节点

#### 为部署设置一个名称，并将模型部署到创建的终端节点。

In [ ]:
DEPLOYED_MODEL_NAME = "[your-deployed-model-name]"  # @param {type: "string"}

# If the deployment name is not set, choose the default one
if DEPLOYED_MODEL_NAME == "[your-deployed-model-name]":
    DEPLOYED_MODEL_NAME = "taxi_fare_prediction_deployment"

# Set the machine type to n1-standard2
MACHINE_TYPE = "n1-standard-2"

# Deploy the model to the endpoint
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=DEPLOYED_MODEL_NAME,
    machine_type=MACHINE_TYPE,
)

model.wait()

print(model.display_name)
print(model.resource_name)

为了确保模型已部署，可以使用 `endpoint.list_models()` 方法来检查已部署模型的ID。

In [ ]:
endpoint.list_models()

从部署的模型中获取解释

为了测试部署的在线模型，请从测试数据中选择两个实例作为有效载荷。

In [ ]:
# format the top 2 test instances as the request's payload
test_json = {"instances": [X_test.iloc[0].tolist(), X_test.iloc[1].tolist()]}

调用带有有效载荷请求的端点，并解析响应以获取解释。解释包括用于训练模型的独立变量的属性，这些属性基于配置的归因方法。在这种情况下，我们使用了“采样Shapely”方法，该方法将结果归因于每个特征，并考虑特征的不同排列。该方法提供了对确切Shapely值的采样近似。有关解释的归因方法的更多信息，请参阅[可解释AI概述](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)。

In [ ]:
features = X_train.columns.to_list()


def plot_attributions(attrs):
    """
    Function to plot the features and their attributions for an instance
    """
    rows = {"feature_name": [], "attribution": []}
    for i, val in enumerate(features):
        rows["feature_name"].append(val)
        rows["attribution"].append(attrs["Input_feature"][i])
    attr_df = pd.DataFrame(rows).set_index("feature_name")
    attr_df.plot(kind="bar")
    plt.show()
    return


def explain_tabular_sample(
    project: str, location: str, endpoint_id: str, instances: list
):
    """
    Function to make an explanation request for the specified payload and generate feature attribution plots
    """
    aiplatform.init(project=project, location=location)

    # endpoint = aiplatform.Endpoint(endpoint_id)

    response = endpoint.explain(instances=instances)
    print("#" * 10 + "Explanations" + "#" * 10)
    for explanation in response.explanations:
        print(" explanation")
        # Feature attributions.
        attributions = explanation.attributions

        for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            print("   output_display_name:", attribution.output_display_name)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                print("   output_index:", output_index)

            plot_attributions(attribution.feature_attributions)

    print("#" * 10 + "Predictions" + "#" * 10)
    for prediction in response.predictions:
        print(prediction)

    return response


test_json = [X_test.iloc[0].tolist(), X_test.iloc[1].tolist()]
prediction = explain_tabular_sample(PROJECT_ID, REGION, endpoint, test_json)

## 下一步

由于芝加哥出租车行程数据集持续更新，每当有新的数据集可用时，可以进行相同类型的分析和模型训练。可以根据数据质量将日期范围从一周延长到一个月甚至更长。除非数据过于嘈杂，否则本笔记本中遵循的大部分步骤仍然有效，并且可以应用到新的数据上。实际上，可以安排笔记本在指定时间运行，使用[Vertex AI Workbench 的执行器](https://console.cloud.google.com/vertex-ai/workbench/list/executions)的调度选项重新训练模型。

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 模型
- 端点
- 云存储桶

In [ ]:
# Undeploy the model
endpoint.undeploy_all()

# Delete the endpoint resource
endpoint.delete()

# Delete the model resource.
model.delete()

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI